In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('../data/merged.csv')

In [7]:
df.shape

(21123, 67)

In [8]:
len(np.unique(df.ruid))

5664

In [4]:
df[:20]

,Unnamed: 0,ruid,visit_id,admit_date,discharge_date,stay_length,n_transfers,readmit_time,readmit_30d,sex,...,gluc,k,na,pcv,plt-ct,systolic,diastolic,bmi,pregnancy_indicator,egfr
0,0,50135262,0,2007-02-08,2007-02-12,4 days 00:00:00.000000000,2,9999 days 00:00:00.000000000,0,F,...,138.0,4.40,136.0,32.0,334.0,140.0,58.0,44.710,0.0,123.677830
1,1,50135262,1,2007-08-03,2007-08-06,3 days 00:00:00.000000000,3,172 days 00:00:00.000000000,0,F,...,213.5,4.45,135.0,39.0,291.5,121.0,61.0,45.025,0.0,89.505000
2,2,50135262,2,2007-08-28,2007-08-29,1 days 00:00:00.000000000,1,22 days 00:00:00.000000000,1,F,...,266.0,3.90,139.0,38.0,308.0,131.0,60.0,46.230,0.0,107.450000
3,3,50135262,3,2008-02-24,2008-02-28,4 days 00:00:00.000000000,2,179 days 00:00:00.000000000,0,F,...,230.0,4.00,135.0,38.0,274.0,151.0,74.0,47.140,0.0,73.010770
4,4,50135262,4,2008-04-12,2008-04-13,1 days 00:00:00.000000000,1,44 days 00:00:00.000000000,0,F,...,162.0,4.10,139.0,36.0,330.0,134.0,66.0,47.360,0.0,84.358415
5,5,50135262,5,2010-10-28,2010-10-29,1 days 00:00:00.000000000,1,928 days 00:00:00.000000000,0,F,...,201.0,3.60,137.5,39.5,264.5,140.0,52.0,47.580,0.0,101.375050
6,6,50135262,6,2011-02-11,2011-02-23,12 days 00:00:00.000000000,4,105 days 00:00:00.000000000,0,F,...,123.0,3.90,139.0,41.0,277.0,144.0,63.0,40.200,0.0,93.491555
7,7,50135262,7,2012-05-23,2012-05-27,4 days 00:00:00.000000000,1,455 days 00:00:00.000000000,0,F,...,243.0,4.00,138.0,39.0,347.0,143.0,63.0,44.250,0.0,59.711960
8,8,50135262,8,2012-09-15,2012-09-22,7 days 00:00:00.000000000,1,111 days 00:00:00.000000000,0,F,...,233.0,3.65,138.5,25.0,245.0,149.0,53.0,42.770,0.0,76.279970
9,9,50135262,9,2014-11-15,2014-11-18,3 days 00:00:00.000000000,2,784 days 00:00:00.000000000,0,F,...,175.0,4.30,135.0,41.0,234.0,131.0,59.0,48.340,0.0,86.953890
